FMCG Sales Data for Time Series Forecasting and Decision Analysis

# Problem Background
 In the fast-paced world of Fast-Moving Consumer Goods (FMCG), understanding sales dynamics can be the difference between a product flying off the shelves or gathering dust. 
 This dataset offers a glimpse into the intricate dance of sales volume, pricing, promotions, and more across various store locations.

# Problem Definition
Use the data to predict sales volume based on other features provided.


# Data 
  https://www.kaggle.com/datasets/krishanukalita/fmcg-sales-demand-forecasting-and-optimization/data




In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('extended_fmcg_demand_forecasting.csv')
df.head()

,Date,Product_Category,Sales_Volume,Price,Promotion,Store_Location,Weekday,Supplier_Cost,Replenishment_Lead_Time,Stock_Level
0,2022-01-01,Household,1583,5.190661,0,Urban,5,9.299281,9,207
1,2022-01-02,Personal Care,1103,8.949596,0,Urban,6,13.274109,5,253
2,2022-01-03,Dairy,455,4.867987,0,Rural,0,13.302265,9,245
3,2022-01-04,Personal Care,1107,16.968596,1,Urban,1,10.056158,5,265
4,2022-01-05,Personal Care,1447,4.309673,1,Rural,2,3.562862,8,334


In [4]:
df.columns

Index(['Date', 'Product_Category', 'Sales_Volume', 'Price', 'Promotion',
       'Store_Location', 'Weekday', 'Supplier_Cost', 'Replenishment_Lead_Time',
       'Stock_Level'],
      dtype='object')